Section 1

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [4]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):

    df_da = (-3)*a**2 - (0.5)*(a**-0.5)
    df_db = (3)*cos(3*b) + (5/2)*(b**1.5)
    df_dc = 1/(c**2)

    return [df_da, df_db, df_dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [12]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
numerical_grad = [0, 0, 0] # TODO
# -----------
## a section
a = 2
b = 3
c = 4
h = 0.00000001


fa1 = f(a, b, c)
fa2 = f(a+h,b,c)

numerical_grad[0] = (fa2-fa1)/h

#b section
fb1 = f(a, b, c)
fb2 = f(a,b+h,c)

numerical_grad[1] = (fb2-fb1)/h

#c section
fc1 = f(a, b, c)
fc2 = f(a,b,c+h)

numerical_grad[2] = (fc2-fc1)/h


print(numerical_grad)
for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

[-12.353553380251014, 10.256990368162633, 0.0624999607623522]
OK for dim 0: expected -12.353553390593273, yours returns -12.353553380251014
OK for dim 1: expected 10.25699027111255, yours returns 10.256990368162633
OK for dim 2: expected 0.0625, yours returns 0.0624999607623522


In [21]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [0, 0, 0] # TODO
# -----------

# -----------
## a section
a = 2
b = 3
c = 4
h = 0.00000001


fa1 = f(a-h, b, c)
fa2 = f(a+h,b,c)

numerical_grad2[0] = (fa2-fa1)/(2*h)

#b section
fb1 = f(a, b-h, c)
fb2 = f(a,b+h,c)

numerical_grad2[1] = (fb2-fb1)/(2*h)

#c section
fc1 = f(a, b, c-h)
fc2 = f(a,b,c + h)

numerical_grad2[2] = (fc2-fc1)/(2*h)

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553291433172
OK for dim 1: expected 10.25699027111255, yours returns 10.256990368162633
OK for dim 2: expected 0.0625, yours returns 0.0624999607623522


## section 2: support for softmax

In [126]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other): # exactly as in the video
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        #set the backward function here
        out._backward = _backward

        return out

    def __mul__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data*other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        #set the backward function here
        out._backward = _backward

        return out

    def tanh(self):
        n = self.data 
        t_val = (exp(2*n) - 1) / (exp(2*n) + 1)
        out = Value(t_val, (self, ), 'tanh')

        def _backward():
            self.grad += out.grad * (1 - t_val**2)

        out._backward = _backward


        return out

    def backward(self): # exactly as in video
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()



# ------
# re-implement all the other functions needed for the exercises below
# your code here
# TODO
# ------


##Manual example of gradient calculation
h = 0.0001
a = Value(2.0, label = 'a')
b = Value(-3.0, label = 'b')
c = Value(10.0, label = 'c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'

f = Value(-2.0, label = 'f')

L = d * f; L.label = 'L'
L1 = L.data


a = Value(2.0, label = 'a')
b = Value(-3.0 + h, label = 'b')
c = Value(10.0, label = 'c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label = 'f')

L = d * f; L.label = 'L'
L2 = L.data

print((L2 - L1)/h)

-4.000000000008441


### Neuron Example

In [127]:
#input values
x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')

#weights
w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')

#bias of the neuron
b = Value(6.88137, label = "b")

x1w1 = x1*w1; label = 'x1*w1'
x2w2 = x2*w2; label = 'x2*w2'

#full input
full_input = x1w1 + x2w2; full_input.label = "x1*w1 + x2*w2"
n = full_input + b; n.label = 'neuron'

o = n.tanh(); o.label = 'output'

In [128]:
o.backward()


In [130]:
n.grad

0.5000025364090595

## Gradient experiment


In [111]:
o.grad = 1.0 

In [112]:
o._backward()
o.grad

1.0

In [113]:
n._backward()
n.grad

0.5000025364090595

In [114]:
b._backward()
b.grad

0.5000025364090595

In [115]:
full_input._backward()
full_input.grad

0.5000025364090595

In [116]:
x2w2._backward()
x2w2.grad

0.5000025364090595

In [117]:
x1w1._backward()
x1w1.grad

0.5000025364090595

In [118]:
x1._backward()
x1.grad

-1.5000076092271784

In [119]:
w1._backward()
w1.grad

1.000005072818119

In [120]:
x2._backward()
x2.grad

0.5000025364090595

In [121]:
w2._backward()
w2.grad

0.0

### Automating backward creation

In [ ]:
def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


In [ ]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
